In [240]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from time import sleep
import os
option = webdriver.ChromeOptions()
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

# INIT

# visit web

In [241]:
import re
"""
ต่อจากด้านบนส่วนนี้จะเป็นการใช้ regular expression ในการสกัด link ออกมา (ขั้นตอนนี้จะไม่ได้ใช้ soup)
r'<a href="(https://www.britannica.com/place/(.*?)) ==> จาก pattern คือการสกัด 
pattern (https://www.britannica.com/place/(กี่ตัวอักษรก็ได้ และมีได้แค่ชุดเดียว) ปล.ลองลบ ? เพื่อความเข้าใจ
"""
url = "https://www.britannica.com/topic/list-of-countries-1993160"
html_data = requests.get(url).text
#name = re.findall(r'<a href="https://www.britannica.com/place/(.*?)" ',str(html_data))
regx_link = re.findall(r'<a href="(https://www.britannica.com/place/(.*?))" ',str(html_data))
links = []
for ele in regx_link:
    link  = ele[0]
    links.append(link)
    

In [242]:
name_of_country = []
info = []
list_flag = []
list_capital = []
list_HeadGoverment  = [] # list ชื่อของ Head of Government หรือคนที่เป็นทั้ง Head of state and government
list_info = []
list_map = []

links = []

In [243]:
def GET_LINK():
    url = "https://www.britannica.com/topic/list-of-countries-1993160"
    html_data = requests.get(url).text
    #name = re.findall(r'<a href="https://www.britannica.com/place/(.*?)" ',str(html_data))
    regx_link = re.findall(r'<a href="(https://www.britannica.com/place/(.*?))" ',str(html_data))
    
    for ele in regx_link:
        link  = ele[0]
        links.append(link)

In [244]:
def GET_FLAGURL(req):
        ## GET FLAG URL
        linkFlag = re.findall('<img loading="lazy" src="(.*?lag.*?)"',req) #flag 2 แบบ Flag flag ทำให้ไม่ matched เลยใช้แค่ (_lag)
        if(linkFlag):
            list_flag.append(linkFlag[0])
        else:
            linkFlag = re.findall('<img loading="lazy" src="(.*?FLAG.*?)"',req)
            if(linkFlag): 
                    list_flag.append(linkFlag[0])
            else:
                linkFlag = re.findall('<img loading="lazy" src="(.*?)"',req) #ใช้กับประเทศ South Sudan เพราะใน link ไม่มีคำว่า flag เลย 
                list_flag.append(linkFlag[1])


In [245]:
def GET_CAPITAL(req):
        nameOfCapital = re.findall('<a href="/place/.*?>(.*?)</a>',req)
        if(nameOfCapital):
            list_capital.append(nameOfCapital[0])
        else:
            nameOfCapital = re.findall('<dt class="d-inline">Capital:</dt>\n\t\t\t\t<dd class="d-inline">\n\t\t\t\t\t<span class="fact-item">(.*?)<',req)
            if(nameOfCapital):
                  list_capital.append(nameOfCapital[0])
            else:
                  list_capital.append("NAN")


In [246]:
def GET_INFO(req):
        rawInfo = re.findall('<p class="topic-paragraph".*?>(.*)</p>',req) 
        if(rawInfo != []):
            SumInfo = ''
            for i in range (len(rawInfo)):
                cleanInfo = re.sub('<.*?>','',rawInfo[i]) #remove html tag
                SumInfo += cleanInfo #sum all info
            list_info.append(SumInfo)
        else:
            print("error")


In [247]:
list_prefix = ['President: ', 'Prime Minister: ','Prime Minister:','Premier: ', 'King: ','Supreme Leader']

In [248]:
def GET_DIRECTOR(req):
        HeadOf = re.findall('Head.*?Government.*:</dt>\n\t\t\t\t<dd class="d-inline">\n\t\t\t\t\t<span class="fact-item">(.*?)<',req) # ใช้ได้กับชื่อที่เป็น tag span ไม่ matched กับ ชื่อที่เป็น tag a
        if(HeadOf != [] and HeadOf[0] not in list_prefix): # ถ้า HeadOf ดึงข้อมูลมาได้แค่ prefix จะไม่เข้าเงื่อนไข  
            list_HeadGoverment.append(HeadOf[0])
        else:
        #ใช้กับชื่อที่มี tag a >> prefix + name
            pre = re.findall('Head.*?Government:</dt>\n\t\t\t\t<dd class="d-inline">\n\t\t\t\t\t<span class="fact-item">(.*?)<',req) #prefix
            name = re.findall('Head.*?Government:</dt>\n\t\t\t\t<dd class="d-inline">\n\t\t\t\t\t<span class="fact-item">.*?: <a href=".*?>(.*?)<',req) #name
            if(HeadOf != [] and pre[0] != '' and name[0] != ''):
                headname = pre[0]+name[0]
                list_HeadGoverment.append(headname)
            else:
                  list_HeadGoverment.append("President: Giuseppe Bertello") # ประเทศ Vatican-City ไม่มีหัวข้อ HeadOf เลยเพิ่มชื่อจากที่หาในวิกิไป


In [249]:
def GET_MAP(req):
    linkmap = re.findall('<img src="(https://cdn.britannica.com/.*?[m|M]ap-.*?)"',req) #re เช็ค link ที่มีคำว่า map หรือ Map
    if(linkmap != []):
        list_map.append(linkmap[len(linkmap)-1]) # มีบางประเทศ re match ได้มาหลาย link แต่ส่วนใหญ่ตัวสุดท้ายใน list จะเป็น link map ที่ต้องการ  
    else:
        usaMap = re.findall('<img src="(https://cdn.britannica.com/.*?The-United-States.*?)"',req) # for USA
        if(usaMap):
            list_map.append(usaMap[len(usaMap) -1])
        else:
            list_map.append("NaN")


In [250]:
GET_LINK()

In [251]:
for url in links:
    r = requests.get(url)
    req = r.text
   
    ## GET NAME OF COUNTRY
    nameOfCountry = re.findall('.*<h1 class="mb-0">(.*)</h1>',req)
    if(nameOfCountry):
        name_of_country.append(nameOfCountry[0])
        
    ## GET FLAG URL
        GET_FLAGURL(req)  
        
    ## GET CAPTITAL
        GET_CAPITAL(req)
    
    ## GET DIRECTOR
        GET_DIRECTOR(req)
                     
    ## GET INFO
        GET_INFO(req)
                     
      ## GET MAP URL
        GET_MAP(req)
        
                     
    else:
        nameList.append("NAN")
        

# PACKING TO DATAFRAME


In [252]:
name_df = pd.DataFrame(name_of_country ,columns=["name"])
capital_df = pd.DataFrame(list_capital ,columns=["capital"])
info_df = pd.DataFrame(list_info ,columns=["info"])
flagUrl_df = pd.DataFrame(list_flag ,columns=["flagUrl"])
sourceUrl_df = pd.DataFrame(links ,columns=["sourceUrl"])
headGoverment = pd.DataFrame(list_HeadGoverment ,columns=["headOfGoverment"])
mapUrl_df = pd.DataFrame(list_map ,columns=["mapUrl"])

pack = [name_df ,capital_df ,info_df,flagUrl_df  ,headGoverment ,mapUrl_df ,sourceUrl_df ]

dataframe = pd.concat(pack, axis=1, join="inner")

In [253]:
dataframe

,name,capital,info,flagUrl,headOfGoverment,mapUrl,sourceUrl
0,Afghanistan,Kabul,"Afghanistan, landlocked multiethnic country l...",https://cdn.britannica.com/40/5340-050-AA46700...,President: Ashraf Ghani; de facto leader Hibat...,https://cdn.britannica.com/76/183576-050-FC2A0...,https://www.britannica.com/place/Afghanistan
1,Albania,Tirana,"Albania, country in southern Europe, located ...",https://cdn.britannica.com/00/6200-050-2E25CE9...,Prime Minister: Edi Rama,https://cdn.britannica.com/77/183577-050-69FDE...,https://www.britannica.com/place/Albania
2,Algeria,Algiers,"Algeria, large, predominantly Muslim country ...",https://cdn.britannica.com/34/3034-050-077DE27...,"President: Abdelmadjid Tebboune, assisted by P...",https://cdn.britannica.com/46/3046-050-87776E4...,https://www.britannica.com/place/Algeria
3,Andorra,Andorra la Vella,"Andorra, small independent European coprincip...",https://cdn.britannica.com/83/5583-050-2F48FD3...,Head of Government: Xavier Espot Zamora,https://cdn.britannica.com/80/183580-050-27967...,https://www.britannica.com/place/Andorra
4,Angola,Luanda,"Angola, country located in southwestern Afric...",https://cdn.britannica.com/12/4212-050-FC69793...,President: João Lourenço,https://cdn.britannica.com/81/183581-050-70E7E...,https://www.britannica.com/place/Angola
...,...,...,...,...,...,...,...
191,Venezuela,Caracas,"Venezuela, country located at the northern en...",https://cdn.britannica.com/04/4904-050-4D71EA8...,President: Nicolás Maduro,https://cdn.britannica.com/06/4906-050-6F6E9B0...,https://www.britannica.com/place/Venezuela
192,Vietnam,Hanoi,"Vietnam, country occupying the eastern portio...",https://cdn.britannica.com/41/4041-050-BB4577F...,Prime Minister: Pham Minh Chinh,https://cdn.britannica.com/51/4051-050-E291105...,https://www.britannica.com/place/Vietnam
193,Yemen,Sanaa,"Yemen, country situated at the southwestern c...",https://cdn.britannica.com/85/5785-050-B3B0056...,Prime Minister: Maeen Abdulmalik Saeed,https://cdn.britannica.com/92/183792-050-72FB6...,https://www.britannica.com/place/Yemen
194,Zambia,Lusaka,"Zambia, landlocked country in south-central A...",https://cdn.britannica.com/31/4231-050-B24F289...,President: Hakainde Hichilema,https://cdn.britannica.com/93/183793-050-F8583...,https://www.britannica.com/place/Zambia


In [254]:
country_data[-1:]

,name,capital,info,flagUrl,headOfGoverment,mapUrl,sourceUrl
391,Zimbabwe,Harare,"Zimbabwe, officially Republic of Zimbabwe, for...",https://cdn.britannica.com/33/4233-050-3A1705E...,President: Emmerson Mnangagwa (interim),https://cdn.britannica.com/45/4245-050-58199B2...,https://www.britannica.com/place/Zimbabwe


# PACK TO JSON

In [255]:
"""
to_json(orient='records') จะ return เป็นแค่ String
การที่จะทำให้เป็น json files จำเป็นต้อง parsed โดยการใช้ json.loads("...json string fomat...")
"""
import json
json_str = dataframe.to_json(orient='records')
parsed = json.loads(json_str)


In [256]:
with open("sample.json", "w" , encoding="utf-8") as outfile:
    json.dump(parsed, outfile, ensure_ascii=False)